In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

In [ ]:
data_train=pd.read_csv('/content/drive/MyDrive/Data Set/classification_train.csv')
data_test=pd.read_csv('/content/drive/MyDrive/Data Set/classification_test.csv')
y0=np.array(data_train[['label']]) 
x0=np.array(data_train.iloc[:, 2:]) 
x_pred=np.array(data_test.iloc[:, 1:])

choice = np.random.choice(range(x0.shape[0]), size=(20000,), replace=False)    
ind = np.zeros(x0.shape[0], dtype=bool)
ind[choice] = True
rest = ~ind
x=x0[ind, :]
x_test=x0[rest, :]
y=y0[ind]
y_test=y0[rest]


In [19]:
def sigmoid(z):
  return 1/(np.exp(-z)+1)

In [20]:
def normalized(x):
  m1=np.mean(x, axis=0)
  sig1=np.std(x, axis=0)
  return (x-m1)/(sig1)

In [21]:
def procedure(y, t):
  y0=np.zeros_like(y)
  y0[y==t]=1
  y0[y!=t]=0
  return y0

In [22]:
def cost(y, y_hat, l):
  m=len(y)
  J=(np.sum(-y*np.log(y_hat)-(1-y)*np.log(1-y_hat))+ np.sum(l*W**2))/(2*m)
  return J

In [23]:
def grad(alpha, W, y, x, iterate_n, l):
  m,n = x.shape 
  J_list=np.zeros(iterate_n)
  p=np.zeros(iterate_n)
  for i in range(iterate_n):
    z=x.dot(W)
    y_hat=sigmoid(z)
    gradients = x.T.dot(y_hat- y) + l*W
    W = W - (1/m) * alpha * gradients
    print(cost(y, y_hat, l))
    J_list[i]=cost(y, y_hat, l)
    p[i]=i
  plt.plot(p, J_list, 'k')
  plt.show()
  return  W

In [ ]:

m,n=np.shape(x)
W=np.zeros((n+1, 10))
x1=np.c_[np.ones(m), normalized(x)]
y1=procedure(y, 0)

for i in range(1, 10):
  y1=np.append(y1, procedure(y, i), axis=-1)

W=grad(0.04, W, y1, x1, 200, 0)
y_train=sigmoid(x1.dot(W))
grp=np.argmax(y_train, axis=-1)

y_t=np.zeros_like(y_train)
for k in range(len(y)):
  y_t[k, grp[k]]=1
tp=0
fp=0
for j in range(10):
  for i in range(len(y)):
    if y1[i, j]==1:
      if y_t[i, j]==y1[i, j]:
        tp+=1
      else:
        fp+=1

print(tp/(tp+fp))

x2=np.c_[np.ones(x_test.shape[0]), normalized(x_test)]
y_hat=sigmoid(x2.dot(W))
grp1=np.argmax(y_hat, axis=-1)
y2=procedure(y_test, 0)
for i in range(1, 10):
  y2=np.append(y2, procedure(y_test, i), axis=-1)

y_o=np.zeros_like(y_hat)
for k in range(len(y_test)):
  y_o[k, grp1[k]]=1
tp=0
fp=0
for j in range(10):
  for i in range(len(y_test)):
    if y2[i, j]==1:
      if y_o[i, j]==y2[i, j]:
        tp+=1
      else:
        fp+=1
print(tp/(tp+fp))

x3=np.c_[np.ones(x_test.shape[0]), normalized(x_pred)]
y_pred=sigmoid(x3.dot(W))
grp2=np.argmax(y_hat, axis=-1)

Predictions=data_test.drop(data_test.columns[[i for i in range(1,785)]], axis=1)
Predictions['Predicted']=grp2
Predictions.to_csv('/content/drive/MyDrive/Data Set/logistic_predictions.csv', index=False)